In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
import sympy as sy
#sys.path.append('/home/victorw/anaconda/envs/dev_pyphs/workspace/lib_pyphs_dev/')
import pyphs

PPTY_PHY_PARAMS = {"positive": True, "real" : True }
PPTY_STATE_VAR  = {"real" : True }

In [3]:
N = 2  # Nombre de tronçons
Nxi = 5        # Nombre d'état par tronçon
N_lambda = N-1 # Nombre de contraintes
Nx = N * Nxi     # Nombre total d'état

In [4]:
nuL_vec    = sy.symbols('nu_L1:{}'.format(N+1), **PPTY_STATE_VAR)
nuR_vec    = sy.symbols('nu_R1:{}'.format(N+1), **PPTY_STATE_VAR)
Pi_y_vec   = sy.symbols('Pi_y1:{}'.format(N+1), **PPTY_STATE_VAR)
rho_vec    = sy.symbols('rho_1:{}'.format(N+1), **PPTY_PHY_PARAMS)
vol_vec    = sy.symbols('V_1:{}'.format(N+1), **PPTY_STATE_VAR)

In [5]:
# Définition: accesseurs pour être en accord avec les notations des documents
def nuL(i):
    return nuL_vec[i-1]
def nuR(i):
    return nuR_vec[i-1]
def Piy(i):
    return Pi_y_vec[i-1]
def rho(i):
    return rho_vec[i-1]
def vol(i):
    return vol_vec[i-1]

## Construire la matrice d'interconnexion pour un tronçon

In [6]:
def Ji(i):
    # INIT
    Nxi = 5
    Ji  = sy.zeros(5)
    ell = sy.symbols('ell_{}'.format(i), **PPTY_PHY_PARAMS)
    L   = sy.symbols('L_{}'.format(i),   **PPTY_PHY_PARAMS)
    Sw  =  ell * L

    Ji[0,2] = -Piy(i)/(rho(i)*vol(i))
    Ji[0,4] = -1/vol(i)
    Ji[1,4] = 1/vol(i)
    Ji[1,2] = Piy(i)/(rho(i)*vol(i))
    Ji[2,3] = -Sw
    Ji[2,4] = Sw*rho(i)/vol(i)
    
    # Antisymétrie
    Ji = Ji -Ji.T
    
    return Ji

Ji(2)

Matrix([
[                0,                  0,   -Pi_y2/(V_2*rho_2),          0,              -1/V_2],
[                0,                  0,    Pi_y2/(V_2*rho_2),          0,               1/V_2],
[Pi_y2/(V_2*rho_2), -Pi_y2/(V_2*rho_2),                    0, -L_2*ell_2, L_2*ell_2*rho_2/V_2],
[                0,                  0,            L_2*ell_2,          0,                   0],
[            1/V_2,             -1/V_2, -L_2*ell_2*rho_2/V_2,          0,                   0]])

In [7]:
# On assemble le J complet, on on a pas permuté les lignes/colonnes
J = sy.SparseMatrix(sy.zeros(Nx))

for i in range(N):
    J[Nxi*i:Nxi*(i+1), Nxi*i:Nxi*(i+1)] = Ji(i+1)

J

Matrix([
[                0,                  0,   -Pi_y1/(V_1*rho_1),          0,              -1/V_1,                 0,                  0,                    0,          0,                   0],
[                0,                  0,    Pi_y1/(V_1*rho_1),          0,               1/V_1,                 0,                  0,                    0,          0,                   0],
[Pi_y1/(V_1*rho_1), -Pi_y1/(V_1*rho_1),                    0, -L_1*ell_1, L_1*ell_1*rho_1/V_1,                 0,                  0,                    0,          0,                   0],
[                0,                  0,            L_1*ell_1,          0,                   0,                 0,                  0,                    0,          0,                   0],
[            1/V_1,             -1/V_1, -L_1*ell_1*rho_1/V_1,          0,                   0,                 0,                  0,                    0,          0,                   0],
[                0,                  0,  

## Permutation
Étant donné un vecteur d'états concaténés $X = [ X_1, X_2, \ldots, X_N]$, on veut permuter les états pour avoir la forme 
$$
X = [\nu_{L1}, \vec{\nu}, \nu_{RN}, \vec{\Pi}_y,  \vec{vol}, \vec{\rho}]
$$


In [8]:
X = sy.zeros(Nx, 1)
for i in range(N):
    Xi = sy.Matrix([nuL(i+1), nuR(i+1), Piy(i+1), vol(i+1), rho(i+1)])
    X[Nxi*i:Nxi*(i+1),0] = Xi
X.T

Matrix([[nu_L1, nu_R1, Pi_y1, V_1, rho_1, nu_L2, nu_R2, Pi_y2, V_2, rho_2]])

On implémente une matrice de permutation générale pour $N$ tronçons

In [9]:
P = sy.SparseMatrix(sy.zeros(Nx))

for i in range(N):
    # nu_L/ nu_R
    P[2*i, Nxi*i]         = 1
    P[2*i+1, Nxi*i+1]     = 1
    # Piy
    P[2*N+i, Nxi*i+2] = 1
    # rho
    P[3*N+i, Nxi*i+3] = 1
    # vol
    P[4*N+i, Nxi*i+4] = 1

P

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [10]:
X2 = P*X
X2.T

Matrix([[nu_L1, nu_R1, nu_L2, nu_R2, Pi_y1, Pi_y2, V_1, V_2, rho_1, rho_2]])

In [11]:
J2 = P*J*P.T
display(J2)

Matrix([
[                0,                  0,                 0,                  0,   -Pi_y1/(V_1*rho_1),                    0,          0,          0,              -1/V_1,                   0],
[                0,                  0,                 0,                  0,    Pi_y1/(V_1*rho_1),                    0,          0,          0,               1/V_1,                   0],
[                0,                  0,                 0,                  0,                    0,   -Pi_y2/(V_2*rho_2),          0,          0,                   0,              -1/V_2],
[                0,                  0,                 0,                  0,                    0,    Pi_y2/(V_2*rho_2),          0,          0,                   0,               1/V_2],
[Pi_y1/(V_1*rho_1), -Pi_y1/(V_1*rho_1),                 0,                  0,                    0,                    0, -L_1*ell_1,          0, L_1*ell_1*rho_1/V_1,                   0],
[                0,                  0, P

## Appliquer le changement de variable
voir notebook 01

In [12]:
bT = sy.zeros(N_lambda, Nx)
for i in range(N_lambda):
    bT[i, 1+2*i] = 1 
    bT[i, 2+i*2] = -1
b = bT.T

annul_b = sy.zeros(Nx-N_lambda,Nx)

annul_b[0,0] = 1
annul_b[N::, N + N_lambda::] = sy.eye(3*N+1)
for i in range(1,N_lambda+1):
    annul_b[i, 2*i-1] = 1 
    annul_b[i, 2*i] = 1
    
M = sy.zeros(Nx, Nx)
M[0:Nx-N_lambda,::] = annul_b
M[Nx-N_lambda::, ::] = (bT*b).inv()*bT

In [13]:
J_cons = M*J2*M.T
display(J_cons)

Matrix([
[                0,                  0,                  0,   -Pi_y1/(V_1*rho_1),                    0,          0,          0,              -1/V_1,                   0,                    0],
[                0,                  0,                  0,    Pi_y1/(V_1*rho_1),   -Pi_y2/(V_2*rho_2),          0,          0,               1/V_1,              -1/V_2,                    0],
[                0,                  0,                  0,                    0,    Pi_y2/(V_2*rho_2),          0,          0,                   0,               1/V_2,                    0],
[Pi_y1/(V_1*rho_1), -Pi_y1/(V_1*rho_1),                  0,                    0,                    0, -L_1*ell_1,          0, L_1*ell_1*rho_1/V_1,                   0, -Pi_y1/(2*V_1*rho_1)],
[                0,  Pi_y2/(V_2*rho_2), -Pi_y2/(V_2*rho_2),                    0,                    0,          0, -L_2*ell_2,                   0, L_2*ell_2*rho_2/V_2, -Pi_y2/(2*V_2*rho_2)],
[                0,       

In [14]:
J_cons_11 = J_cons[0:Nx-N_lambda-1, 0:Nx-N_lambda-1]

In [15]:
J_cons_11

Matrix([
[                0,                  0,                  0,   -Pi_y1/(V_1*rho_1),                  0,          0,          0,              -1/V_1],
[                0,                  0,                  0,    Pi_y1/(V_1*rho_1), -Pi_y2/(V_2*rho_2),          0,          0,               1/V_1],
[                0,                  0,                  0,                    0,  Pi_y2/(V_2*rho_2),          0,          0,                   0],
[Pi_y1/(V_1*rho_1), -Pi_y1/(V_1*rho_1),                  0,                    0,                  0, -L_1*ell_1,          0, L_1*ell_1*rho_1/V_1],
[                0,  Pi_y2/(V_2*rho_2), -Pi_y2/(V_2*rho_2),                    0,                  0,          0, -L_2*ell_2,                   0],
[                0,                  0,                  0,            L_1*ell_1,                  0,          0,          0,                   0],
[                0,                  0,                  0,                    0,          L_2*ell_2,  

## Autres matrices
$J_{yy}$ est remplie de 0. $J_{xy}$ est assez facile à coder: 1 port à gauche, 1 port à droite; un port par paroi soit N ports. Au total, $N+2$ ports soit:
$$
Y = \left[\begin{array}{c}
 -q_L \\
 q_R \\
 -v_{w1} \\
 \vdots \\
 -v_{wN}
 \end{array}\right]
$$

In [28]:
Jyx = sy.SparseMatrix(sy.zeros(N+2, Nx))
Jyx[0,0] = -1
Jyx[1, 2*N-1] = 1
Jyx[2::,2*N:3*N] = - sy.eye(N)
Jyx

Matrix([
[-1, 0, 0, 0,  0,  0, 0, 0, 0, 0],
[ 0, 0, 0, 1,  0,  0, 0, 0, 0, 0],
[ 0, 0, 0, 0, -1,  0, 0, 0, 0, 0],
[ 0, 0, 0, 0,  0, -1, 0, 0, 0, 0]])